### Import

In [71]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import metrics
from sklearn import neighbors
from sklearn import svm
from sklearn import decomposition
from sklearn.decomposition import PCA

### Load data

In [72]:
def load_data():
    # this_directory = os.path.dirname(os.path.abspath(__file__))
    data = pd.read_csv(os.path.join('hn', 'HN_radiomicFeatures.csv'), index_col=0)

    return data


data = load_data()
label = data["label"]
data = data.drop(["label"], axis=1)
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

The number of samples: 113
The number of columns: 159


### Check for missing data

In [73]:
# Detect missing values in dataframe
data.isnull().values.any()

False

### Exclude features with zero variance

In [74]:
# exclude features with zero variances
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold()
selector.fit_transform(data)
selector.fit_transform(data).shape

(113, 157)

### Data split

In [75]:
train_val_data, test_data, train_val_label, test_label = train_test_split(data, label, train_size=0.8)
train_data, val_data, train_label, val_label = train_test_split(train_val_data, train_val_label, train_size=0.85)

print(f'The number of train samples: {train_data.shape[0]}')
print(f'The number of validation samples: {val_data.shape[0]}')
print(f'The number of test samples: {test_data.shape[0]}')

The number of train samples: 76
The number of validation samples: 14
The number of test samples: 23


### Scaling train data

In [76]:
scaler = StandardScaler().fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

### Scaler & Linear Classifier

In [77]:
clf = SGDClassifier()
clf.fit(train_data, train_label)
print(np.shape(clf.coef_))
print(clf.intercept_)

label_train_pred = clf.predict(train_data)
print(metrics.accuracy_score(train_label, label_train_pred))

y_pred = clf.predict(test_data)
print(metrics.accuracy_score(test_label, y_pred))

(1, 159)
[-107.71985976]
0.9605263157894737
0.782608695652174


### kNN classifier

In [78]:
# Perform a PCA
pca = decomposition.PCA(n_components=10)
pca.fit(train_data)
train_pca = pca.transform(train_data)
test_pca = pca.transform(test_data)
print(sum(pca.explained_variance_ratio_))

# Fit kNN
knn = neighbors.KNeighborsClassifier(n_neighbors=10)
knn.fit(train_pca, train_val_label)
score_train = knn.score(train_pca, train_val_label)
score_test = knn.score(test_pca, test_label)

# Print result
print(f"Training result: {score_train}")
print(f"Test result: {score_test}")

0.730353362550487


ValueError: Found input variables with inconsistent numbers of samples: [76, 90]